# Query Expansion

0) Just some imports

In [1]:
import re
import math
import numpy as np
import common as cm

# 1) Simple search engine

1.1) Get acquainted with the below class. There are several TODOs. However, DO NOT complete them now. 

In [2]:
class Dictionary:
    def __init__(self):
        ### keeps unique terms (SORTED!)
        self.terms = self.loadTerms("terms.txt")
        self.idfs = [] ### IDF coefficients
        self.corM = [] ### a correlation matrix

    ### load terms
    def loadTerms(self, fileName):
        file = open(fileName,'r', encoding='utf-8-sig')
        k = [self.proc(s) for s in file.readlines()]
        k.sort()
        return k

    ### ignore it
    def proc(self, s):
        if s[-1] == '\n': return s[:-1]
        else: return s
    
    ### TODO (DO NOT FINISH THIS METHOD YET. YOU WILL BE ASKED FOR IT LATER) 
    # DONE
    def computeIDFs(self, documents):
        self.idfs = []
        for term in self.terms:
            occurences = sum([doc.tokens.count(term) for doc in documents])
            self.idfs.append(math.log2(len(documents) / occurences))
    
    ### TODO (DO NOT FINISH THIS METHOD YET. YOU WILL BE ASKED FOR IT LATER) 
    # DONE
    def computeCorM(self, documents):
        self.corM = []
        for doc1 in documents:
            row = []
            for doc2 in documents:
                a = set(doc1.tokens).intersection(set(doc2.tokens))
                t = set(doc1.tokens).union(set(doc2.tokens))
                if doc1 == doc2:
                    row.append(-1.0)
                else:
                    row.append(len(a)/len(t))
            self.corM.append(row)
        

### SOME DEBUG
dictionary = Dictionary()
print(dictionary.terms[:50])

['aaai', 'about', 'academic', 'access', 'acquired', 'acquisition', 'action', 'activity', 'actual', 'adaptive', 'add', 'advance', 'agricultural', 'aha', 'aim', 'alert', 'algorithm', 'all', 'analysis', 'and', 'announcement', 'answer', 'anyone', 'application', 'applied', 'apply', 'applying', 'approach', 'approache', 'april', 'archive', 'are', 'area', 'areas', 'article', 'artificial', 'asked', 'august', 'author', 'automated', 'automatically', 'autonomous', 'available', 'awards', 'backend', 'backgammon', 'baldi', 'based', 'basic', 'bayesian']


1.2) Load files: here we load some example collection of documents. RAW_DOCUMENTS = just strings. Check if the documents are loaded correctly (e.g., print RAW_DOCUMENTS[0])

In [3]:
RAW_DOCUMENTS = cm.loadDocuments("documents.txt")
### SOME DEBUG
print(RAW_DOCUMENTS[0])

David W. Aha:  Machine Learning Page
 Machine Learning Resources. Suggestions welcome. ... (WizRule); ZDM Scientific
 Ltd. Conference Announcements. Courses on Machine Learning. Data Repositories. ... 
 Description: Comprehensive machine learning resources from Applications to Tutorials.



In [4]:
### SOME DEBUG, JUST RUN; check if (a) common.py is imported correctly and (b) 
### tokens are correctly derived from some document (e.g., RAW_DOCUMENTS[0])
print(cm.simpleTextProcessing(RAW_DOCUMENTS[0], re))

['david', 'aha', 'machine', 'learning', 'page', 'machine', 'learning', 'resource', 'suggestion', 'welcome', 'wizrule', 'zdm', 'scientific', 'ltd', 'conference', 'announcement', 'course', 'machine', 'learning', 'data', 'repository', 'description', 'comprehensive', 'machine', 'learning', 'resource', 'from', 'application', 'tutorials']


1.3) Get acquainted with the below class. 

In [5]:
class Document:
    def __init__(self, doc_id, raw_document, dictionary):
        self.doc_id = doc_id ### DOC ID, simply 0,1,2,3....
        self.raw_document = raw_document ### raw data, i.e., string data
        self.dictionary = dictionary # reference to the dictionary
        
        ### DOCUMENT REPRESENTATIONS
        self.tokens = cm.simpleTextProcessing(raw_document, re) ### get terms
        self.bow = [] # Bag Of Words (BOW - number of term occurences)
        self.tf = [] # TF representation
        self.tf_idf = [] # TF-IDF representation

    ### TODO - complete this method; it should compute a BOW representation
    ## DONE: implemented version with number of occurences
    def computeBOW_Representation(self):
        self.bow = []
        for term in self.dictionary.terms:
            self.bow.append(self.tokens.count(term))
    
    ### TODO - complete this method; it should compute a TF representation
    # DONE
    def computeTF_Representation(self):
        self.tf = []
        max_number = max(self.bow)
        for number in self.bow:
            self.tf.append(number / max_number)
    
    ### TODO - complete this method; it should compute a TFxIDF representation 
    ### (important: it should not be run before dictionary.idfs are not computed!)
    # DONE
    def computeTF_IDF_Representation(self):
        self.tf_idf = []
        for i, idf in enumerate(self.dictionary.idfs):
            self.tf_idf.append(idf * self.tf[i])
    
    def computeRepresentations(self):
        self.computeBOW_Representation()
        self.computeTF_Representation()
        self.computeTF_IDF_Representation()
    
documents = [Document(i, RAW_DOCUMENTS[i], dictionary) for i in range(len(RAW_DOCUMENTS))]

1.4) Compute IDFs here

In [6]:
### TODO COMPUTE IDFS HERE (FINISH THE PROPER METHOD OF THE DICTIONARY CLASS - DO NOT FORGET TO RE-RUN THE CELL)
dictionary.computeIDFs(documents)

### SOME DEBUG
res = [[dictionary.terms[i], dictionary.idfs[i]] for i in range(len(dictionary.terms))]
res.sort(key = lambda x: x[1])
# LEAST COMMON WORDS - HIGH IDF
print(res[-5:])
# MOST COMMON WORDS - LOW IDF
print(res[:5])

[['working', 6.459431618637297], ['www', 6.459431618637297], ['york', 6.459431618637297], ['young', 6.459431618637297], ['zdm', 6.459431618637297]]
[['learning', -1.3797721694596468], ['machine', -1.3479233034203069], ['the', -0.14048122354983045], ['and', 0.11958161575267245], ['for', 1.1375035237499351]]


1.5) Compute the document representations (for each document run computeRepresentations())

In [7]:
for d in documents: d.computeRepresentations()
### SOME DEBUG (you should see some 4s - which terms are these?)
print(documents[0].bow[:30])
print(documents[0].tf_idf[:30])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.1148579046593243, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.3648579046593243, 0.0, 0.0, 0.8223766542987463, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


1.6) Finish the below method. It should compute and return a cosine similarity (v1 and v2 are two vectors - tf-idf representations)

In [8]:
### TODO 
# DONE
def getSimilarity(v1, v2):
    numerator, denominatorV1, denominatorV2 = 0, 0, 0
    for i in range(len(v1)):
        numerator += v1[i] * v2[i]
        denominatorV1 += v1[i] * v1[i]
        denominatorV2 += v2[i] * v2[i]
    return numerator / math.sqrt(denominatorV1 * denominatorV2)

1.7) Run the below script for different queries. getTopResults seeks for documents being the most similar/relevant to the query. Do you find the results satisfactory?

In [9]:
#query = "machine learning"
#query = "academic research"
#query = "international conference"
query = "international conference washington"

In [10]:
def getTopResults(query, documents, dictionary, similarity, top = 5):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults(query, documents, dictionary, getSimilarity, top = 5)

RANK = 1 WITH SIMILARITY = 0.4616849938054148 | DOC ID = 76
ICML 2003
 The Twentieth International Conference on Machine Learning (ICML-2003).
 August 21-24, 2003 Washington, DC USA. The Twentieth International ... 


RANK = 2 WITH SIMILARITY = 0.2984957010826983 | DOC ID = 86
International Conferences on Machine Learning and Applications
 The 2002 International Conferences on Machine Learning and Applications The
 2003 International Conferences on Machine Learning and Applications. 


RANK = 3 WITH SIMILARITY = 0.18888157624160318 | DOC ID = 19
1998 International Machine Learning Conference
 (larger version of cover). The Fifteenth International Conference on Machine Learning.
 The on-line schedule for ICML-98 contains links to many of the papers. ... 
 Description: The Fifteenth International Conference on Machine Learning. July 24-27, 1998 in Madison, Wisconsin.


RANK = 4 WITH SIMILARITY = 0.1379014811828771 | DOC ID = 41
ICML-2000
 Seventeenth International Conference on Machine L

# 2) Query expansion

## 2.1) Correlation matrix

2.1.1) Finish dictionary.computeCorM method (see class Dictionary). It should generate a correlation matrix (correlation between terms).

IMPORTANT: although corM[ i ][ i ] (for each i) should be 1.0, set it to -1.0

In [11]:
### TODO - COMPLETE THE computeCorM METHOD (see one of the first cells)
dictionary.computeCorM(documents)
print(dictionary.corM)

[[-1.0, 0.15, 0.15789473684210525, 0.06060606060606061, 0.08888888888888889, 0.08571428571428572, 0.08108108108108109, 0.12903225806451613, 0.08333333333333333, 0.06976744186046512, 0.02631578947368421, 0.08333333333333333, 0.11428571428571428, 0.06976744186046512, 0.05714285714285714, 0.08571428571428572, 0.11428571428571428, 0.09375, 0.08695652173913043, 0.10526315789473684, 0.0625, 0.1282051282051282, 0.06666666666666667, 0.06818181818181818, 0.08108108108108109, 0.05555555555555555, 0.1282051282051282, 0.08695652173913043, 0.0625, 0.125, 0.13513513513513514, 0.09090909090909091, 0.05714285714285714, 0.08823529411764706, 0.1724137931034483, 0.12903225806451613, 0.08108108108108109, 0.16666666666666666, 0.07894736842105263, 0.10714285714285714, 0.08823529411764706, 0.22580645161290322, 0.1111111111111111, 0.05263157894736842, 0.08695652173913043, 0.07407407407407407, 0.05555555555555555, 0.1, 0.12195121951219512, 0.0967741935483871, 0.08108108108108109, 0.10638297872340426, 0.0888888

2.1.2) Finish the below method. For each term in the query (you must parse the query, see getTopResults() method), find another term which is the most correlated with the input term.

In [12]:
# query = "machine"
# query = "algorithm"
query = "learning"
# query = "conference"
# query = "research"
# query = "concept"

def suggestKeywords(query, dictionary):
    ### TODO
    # DONE
    print("Suggestions:")
    
    matrix = []
    for i, term in enumerate(dictionary.terms):
        # get number of each term occurences in documents and sum
        row = [ doc.bow[i] for doc in documents ]
        rowSum = sum([ doc.bow[i]**2 for doc in documents])
        row = [i / (rowSum ** (1 / 2)) for i in row]
        matrix.append(row)
        
    matrix = np.asarray(matrix)
    finalM = matrix@matrix.T
    
    index = dictionary.terms.index(query)
    for i in range(len(finalM)):
        finalM[i, i] *= -1.0
        
    m = max(finalM[index])
    mostCorrelated = 0
    for i in range(len(finalM[index])):
        if finalM[index, i] == m:
            mostCorrelated = i
    
    print(dictionary.terms[mostCorrelated])
        
suggestKeywords(query, dictionary)

Suggestions:
machine


# 2.2) Rocchio algorithm

$\overrightarrow{q_{m}} = \alpha \overrightarrow{q} + \left(\beta \cdot \dfrac{1}{|D_{r}|} \sum_{\overrightarrow{D_j} \in D_{r}} \overrightarrow{D_j} \right) - \left(\gamma \cdot \dfrac{1}{|D_{nr}|} \sum_{\overrightarrow{D_j} \in D_{nr}} \overrightarrow{D_j} \right)$

2.2.1) Firstly, run the below code. Observe the results. Assume that we do not like the first and the second result (Docs 63 and 77). However, assume that docs 29 and 36 are satisfactory. Now, modfify the method. It should alter the query vector, according to Rocchio algorithm. Check the result for the above considered scenario (relevant docs = 29 and 36; not relevant = 63 and 77). Check the results for different values of alpha, beta, and gamma coefficients. 

In [21]:
def getTopResults_Rocchio(query, 
                          documents, 
                          dictionary, 
                          similarity, 
                          rel_docs = [29, 36],
                          nrel_docs = [63, 77],
                          alpha = 0.5,
                          beta = 0.3,
                          gamma = 0.2,
                          top = 10):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ##### TODO: MODIFY qd.tf_idf HERE
    # DONE: needs check

    A = [alpha * i for i in qd.tf_idf]
    bSum = [0 for i in documents[0].tf_idf]
    
    for i in rel_docs:
        for b in range(len(bSum)):
            bSum[b] += documents[i].tf_idf[b]
            
    qd.tf_idf = bSum
    
    #####
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults_Rocchio("machine learning", documents, dictionary, getSimilarity, top = 10)

RANK = 1 WITH SIMILARITY = 0.8256488742012702 | DOC ID = 29
Machine Learning
 6.858/18.428: Machine Learning. ... This course deals with the following topics:
 Formal models of machine learning; Learning concepts from examples; ... 


RANK = 2 WITH SIMILARITY = 0.733111425632266 | DOC ID = 36
Machine Learning Group
 [ Bristol CS | Index | Research | ML group | Student projects ] Machine
 Learning Research Group. Overview. The Machine Learning Research ... 
 Description: Research on higher-order concept learning, inductive logic programming, multi-agent learning systems,...


RANK = 3 WITH SIMILARITY = 0.337336032623368 | DOC ID = 81
Oxford University Machine Learning Group
 Machine Learning at the Computing Laboratory. ... Logic Programming and
 Learning and Intelligent Systems. Other Machine Learning Groups. ... 


RANK = 4 WITH SIMILARITY = 0.3296147093692282 | DOC ID = 13
UTCS Machine Learning Research Group
 Machine learning is the study of adaptive computational systems that
 impr

# 2.3) WordNet

2.3.1) Installation

http://www.nltk.org/install.html

import nltk 

nltk.download()

https://www.nltk.org/data.html

In [14]:
from nltk.corpus import wordnet as wn

Definition: synset = (from wiki) (information science) A set of one or more synonyms that are interchangeable in some context without changing the truth value of the proposition in which they are embedded.

2.3.2) Display sysents for "machine"

In [15]:
wn.synsets('machine')

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - 'C:\\Users\\jansmi/nltk_data'
    - 'C:\\Users\\jansmi\\Anaconda3\\nltk_data'
    - 'C:\\Users\\jansmi\\Anaconda3\\share\\nltk_data'
    - 'C:\\Users\\jansmi\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\jansmi\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


2.3.3) Display all definitions (.definition()) for synsets (machine)

In [ ]:
#TODO

2.3.4) For each synset (machine), display its hypernym (a word with a broad meaning constituting a category into which words with more specific meanings fall; a superordinate. For example, colour is a hypernym of red).

In [ ]:
#TODO

See: http://www.nltk.org/howto/wordnet.html
for more examples